In [1]:
from src.bootstrap_error_analysis import *
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
%matplotlib ipympl

In [7]:
ANALYSIS_BASE = "/Volumes/Analysis/"
gsort_path = "/Volumes/Scratch/Users/praful/triplet_gsort_v2_30um_raphe-affinity_cosine"
dataset = "2020-09-29-2"
estim = "data007/data007-all"
wnoise = "kilosort_data002/data002"
thr_factor = 1
p = 1
n = 124


# ANALYSIS_BASE = "/Volumes/Analysis/"
# gsort_path = "/Volumes/Scratch/Users/ajphillips/papers/temporal_dithering_shah/gsort_v2"
# dataset = "2022-02-08-1"
# estim = "data012"
# wnoise = ""
# thr_factor = 1
# p = 103
# n = 4237

In [11]:
relevant_movies, total_probs, edge_probs, clusters, edges, electrodes = get_bootstrap_information(ANALYSIS_BASE, gsort_path,dataset, estim, wnoise,p,n)

  0%|                                                                                                                                                                                | 0/40 [00:00<?, ?it/s]


KeyError: 'graph_info'

In [12]:
electrical_path = os.path.join(ANALYSIS_BASE, dataset, estim)
filepath = os.path.join(gsort_path, 
                        dataset, estim, wnoise, "p" + str(p))

relevant_movies = [f for f in os.listdir(filepath) if f"_n{n}_" in f and "residual" not in f]
num_movies = len(relevant_movies)

total_probs = np.zeros(num_movies)
edge_probs = []
clusters = []
edges = []
for k, movie in enumerate(tqdm.tqdm(relevant_movies)):
    # if k%1000 == 0:
    #     print(f"{k} out of {num_movies}")
    with open(os.path.join(filepath,movie), "rb") as f:
        prob_dict = pickle.load(f)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 51.85it/s]


In [13]:
prob_dict

{'prob': 0.44,
 'cosine_prob': (0.4, 0.7),
 'edge_probs': array([0.28, 0.12, 0.04]),
 'non_saturated_template': array([1., 1., 1., 1., 1.]),
 'data_similarity': array([0.91600872, 0.86914049, 0.67784145]),
 'resolved': True,
 'initial_clustering': array([4, 3, 0, 0, 0, 2, 1, 3, 0, 3, 0, 2, 3, 2, 2, 0, 3, 4, 0, 3, 3, 2,
        2, 4, 2]),
 'initial_clustering_with_virtual': array([4, 3, 0, 0, 0, 2, 1, 3, 0, 3, 0, 2, 3, 2, 2, 0, 3, 4, 0, 3, 3, 2,
        2, 4, 2]),
 'cell_in_clusters': {4237: [3, 4, 1], 4311: [0, 3]},
 'final_clustering': array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2]),
 'iterations': 0,
 'num_trials': 25,
 'note': ''}

In [ ]:
make_surface_plot(relevant_movies, total_probs, ANALYSIS_BASE,dataset, estim, p, n)

In [ ]:
dss, dss_stack, num_electrodes, num_samples, movie_stack= get_difference_signals(relevant_movies,clusters,edges,ANALYSIS_BASE, dataset, estim,p, electrodes)

In [ ]:
plt.figure()
ax = plt.subplot(111)
ax.plot(dss_stack[:100,0,:].T, color = "C0")
plt.show()

In [ ]:
average_ds, average_error_stack,std_error_stack, shifted_dss, shifted_dss_stack,error, error_stack = compute_average_diff_signal_and_error(dss,dss_stack, num_electrodes, num_samples)

In [ ]:
plt.figure()
ax = plt.subplot(212)
ax.plot(shifted_dss_stack[error_stack[:,0]> average_error[0],0,:].T, color = "C0")
ax.plot(average_ds[0,:].T, color = "C1")
ax.set_title("Bad spikes")
ax = plt.subplot(211)
ax.plot(shifted_dss_stack[error_stack[:,0]< average_error[0],0,:].T, color = "C0")
ax.plot(average_ds[0,:].T, color = "C1")

ax.set_title("Good spikes")
per_good_spikes = np.mean(error_stack[:,0]< average_error[0])
plt.suptitle(f"% good spikes = {per_good_spikes}")
plt.show()

In [ ]:
mean_edge_error = get_mean_edge_error(error)

In [ ]:
new_edge_probs = get_modified_edge_probs(mean_edge_error, edge_probs,average_error)

In [ ]:
new_total_probs = np.array([np.sum(p) for p in new_edge_probs])

In [ ]:
plt.figure()
ax = plt.subplot(111)
ax.hist(new_total_probs-total_probs, bins = 20);

In [ ]:
make_surface_plot(relevant_movies, new_total_probs, ANALYSIS_BASE,dataset, estim, p, n)